In [1]:
%matplotlib inline

# PLANE OF SKY (POS) MAGNETIC FIELD STRENGTH II
Author: Jordan Guerra (for Villanova University). January 2024.

This tutorial illustrate the use of some advanced functions of the python package *polBpy* for calculating the POS magnetic field strength using different Davis-Chandrasekhar-Fermi (DCF) approximations.

This tutorial uses data from literature listed [here.](https://github.com/jorgueagui/polBpy/blob/9039d4af5d25c49130bf51be7fe0ce363424edcc/refs.md)

**EXAMPLE I**: This example shows how to calculate statistical (median, 5th, 95th) values of $B_{POS}$
  using the classical DCF approximation. We reproduce some results from Guerra et. al. (2023), which presents results of the CND using observations of SOFIA/HAWC+.

In [2]:
import os
os.chdir(os.path.expanduser("/Users/jguerraa/Desktop/polBpy"))
from polBpy import DCF
import numpy as np

First, we calculate the $B_{POS}$ for the CND using the 53 $\mu$m data and the classical DCF approximation. In this case, we use maps (rather than single values) colunm density and velocity dispersion but a single-value polarization angle dispersion (Table 5). We read these valuess from a FITS file

In [3]:
from astropy.io import fits
file = "/Users/jguerraa/Desktop/polBpy/tutorials/PolBpy_Tutorial_II_data.fits"
data = fits.open(file)
print(data.info())

Filename: /Users/jguerraa/Desktop/polBpy/tutorials/PolBpy_Tutorial_II_data.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  STOKES I      1 PrimaryHDU     566   ()      
  1  COL_DEN       1 ImageHDU         8   (122, 126)   float64   
  2  VEL_DISP      1 ImageHDU         8   (122, 126)   float64   
  3  POS_VEL       1 ImageHDU         8   (122, 126)   float64   
  4  POS_VEL_ERROR    1 ImageHDU         8   (122, 126)   float64   
None


In [4]:
col_den = data['COL_DEN'].data # cm^-2
vel_disp = data['VEL_DISP'].data # km/s

While the cloud's depth and angular dispersion can be single values

In [5]:
cloud_depth = 3.38E+18 # cm
ang_disp = 0.48 # rad

Now we call DCF_range 

In [6]:
bpos = DCF.dcf_range(col_den,vel_disp,ang_disp,rho=False,m_cdepth=cloud_depth)

Variable:  Pol Angle dispersion is a single value. Creating an array with this value.


The result of this calculation is three numbers: 5, 50, and 95 percentiles of the Bpos distribution of values.

In [7]:
print("Median Bpos = %2.2f [mG]"%(bpos[1]/1000.))

Median Bpos = 7.37 [mG]


In [8]:
print("5-percentile Bpos = %2.2f [mG]"%(bpos[0]/1000.))

5-percentile Bpos = 5.39 [mG]


In [9]:
print("95-percentile Bpos = %2.2f [mG]"%(bpos[2]/1000.))

95-percentile Bpos = 10.00 [mG]


Now including the uncertainties of the used variables

In [10]:
ucol_den = col_den.copy()
ucol_den[:] = 0.0
uvel_disp = vel_disp.copy()
uvel_disp[:] = 0.0
uang_disp = 0.07
ucloud_depth = 0.51E+18

In [11]:
bpos = DCF.dcf_range(col_den,vel_disp,ang_disp,rho=False,m_cdepth=cloud_depth,m_uden=ucol_den,m_uvel=uvel_disp,m_udisp=uang_disp,m_ucdepth=ucloud_depth)

Variable:  Pol Angle dispersion is a single value. Creating an array with this value.


In this case, each list element is a tuple in the form (median,sigma):

In [12]:
print("Median Bpos = %2.2f +/- %2.2f [mG]"%(bpos[1][0]/1000.,bpos[1][1]/1000.))

Median Bpos = 7.37 +/- 1.21 [mG]


In [13]:
print("5-percentile Bpos = %2.2f +/- %2.2f [mG]"%(bpos[0][0]/1000.,bpos[0][1]/1000.))

5-percentile Bpos = 5.40 +/- 0.89 [mG]


In [14]:
print("95-percentile Bpos = %2.2f +/- %2.2f [mG]"%(bpos[2][0]/1000.,bpos[2][1]/1000.))

95-percentile Bpos = 10.00 +/- 1.64 [mG]


**EXAMPLE II**: This example shows how to calculate statistical (median, 5th, 95th) values of $B_{POS}$
  using the large-scale and shear flow DCF approximation. We reproduce some results from Guerra et. al. (2023), which presents results of the CND using observations of SOFIA/HAWC+.